In [1]:
import json
import os
import sys
import plotly.express as px
from dash import Dash, html, dcc, Input, Output, callback

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from src.ingest import aimlabs_task_data_to_dfs

In [2]:
fp = "C:\\Users\\hikaru\\Desktop\\aim\\AimlabTaskData_2023-07-26.04.22.24\\taskData.json"

with open(fp, 'r') as f:
    task_data = json.load(f)

data_df, perf_df = aimlabs_task_data_to_dfs(task_data)

In [3]:
data_df.head()

,taskId,taskName,score,mode,performance,startedAt,endedAt
0,331,1_strafe_track,525,42,"{""shotsTotal"":2428,""hitsTotal"":525.0,""missesTo...",2023-07-09 23:37:29,2023-07-09 23:38:55
1,332,1_strafe_track,1359,42,"{""shotsTotal"":5991,""hitsTotal"":1359.0,""missesT...",2023-07-09 23:39:18,2023-07-09 23:40:18
2,333,1_strafe_track,1578,42,"{""shotsTotal"":5984,""hitsTotal"":1578.0,""missesT...",2023-07-09 23:41:49,2023-07-09 23:42:49
3,334,2_adjust_track,1984,42,"{""shotsTotal"":5998,""hitsTotal"":1984.0,""missesT...",2023-07-09 23:44:00,2023-07-09 23:45:01
4,335,2_adjust_track,1873,42,"{""shotsTotal"":5992,""hitsTotal"":1873.0,""missesT...",2023-07-09 23:45:19,2023-07-09 23:46:19


In [4]:
perf_df.head()

,taskId,shotsTotal,hitsTotal,missesTotal,avgDist,damageTotal,timePerKill,killTotal,targetsTotal,accTotal,headshots,bodyshots
0,331,2428,525.0,1903.0,7.899969,0.0,0.0,0,2,21.676300,0,525
1,332,5991,1359.0,4632.0,8.581873,0.0,0.0,0,2,22.684025,0,1359
2,333,5984,1578.0,4406.0,7.816439,0.0,0.0,0,2,26.445452,0,1578
3,334,5998,1984.0,4014.0,11.436398,0.0,0.0,0,39,33.105290,0,1984
4,335,5992,1873.0,4119.0,11.400455,0.0,0.0,0,37,31.258345,0,1873


In [5]:
data_mask = (data_df['taskName'] == data_df['taskName'].unique()[0])
fig = px.scatter(data_df[data_mask], x='startedAt', y='score')
fig.show()

In [20]:
app = Dash(__name__)

app.layout = html.Div([
    html.Div([
        dcc.Graph(
            id='graph1'
        ),
        dcc.RadioItems(
            ['lin', 'log'], 'lin',
            id='options', inline=True
        )
    ], style={'width':'49%'}),
    # html.Div([
    #    dcc.Graph(
    #        id='graph1'
    #    ) 
    # ], style={'width':'49%'})
])

@callback(
    Output('graph1', 'figure'),
    Input('options', 'value')
)
def update_graph(value):
    data_mask = (data_df['taskName'] == data_df['taskName'].unique()[0])
    fig = px.scatter(data_df, x='startedAt', y='score')
    # fig.add_annotation(x=1, y=1, text='hello')
    return fig

In [21]:
app.run(jupyter_mode='inline')